[Lecture 7: Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy)

In [1]:
import requests
from pathlib import Path

data_path = Path("../../data/shakespeare.txt")
if not data_path.exists():
    text = requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt").text
    with open(data_path, mode="w") as f:
        f.write(text)
else:
    with open(data_path, mode="r") as f:
        text = f.read()

print(len(text))
print(text[:300])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [2]:
chars = sorted(list(set(text)))
print("".join(chars))

VOCAB_SIZE = len(chars)
print(VOCAB_SIZE)

ctoi = {c:i for (i,c) in enumerate(chars)}
itoc = {i:c for (i,c) in enumerate(chars)}

def encode(string):
    return [ctoi[c] for c in string]

def decode(inds):
    return "".join([itoc[i] for i in inds])

print(encode("Hello world!"))
print(decode(encode("Hello world!")))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [3]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

n_train = int(0.9 * len(data))
train_data = data[:n_train]
val_data = data[n_train:]

print(train_data.shape, val_data.shape)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
torch.Size([1003854]) torch.Size([111540])


In [4]:
torch.manual_seed(1337)

BLOCK_SIZE = 8
BATCH_SIZE = 4

def get_batch(data: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    idxs = torch.randint(0, len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    xb = torch.stack([data[i:i+BLOCK_SIZE] for i in idxs])
    yb = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in idxs])
    return xb, yb

xb, yb = get_batch(train_data)
print(xb.shape, yb.shape)
print(xb)
print(yb)

for t in range(BLOCK_SIZE):
    print(f"{xb[0, :t+1]} -> {yb[0,t]}")

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
tensor([24]) -> 43
tensor([24, 43]) -> 58
tensor([24, 43, 58]) -> 5
tensor([24, 43, 58,  5]) -> 57
tensor([24, 43, 58,  5, 57]) -> 1
tensor([24, 43, 58,  5, 57,  1]) -> 46
tensor([24, 43, 58,  5, 57,  1, 46]) -> 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) -> 39


In [5]:
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.optim import AdamW

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)

    def forward(self, xb, yb=None):
        logits = self.embedding(xb)

        if yb is None:
            loss = None
        else:
            B,T,C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), yb.view(B*T))
        return logits, loss
    
    def generate(self, xb, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(xb)
            logits = logits[:, -1, :]  # B,T,C -> B, C
            probs = F.softmax(logits, dim=-1)
            new_inds = torch.multinomial(probs, num_samples=1)  # B
            xb = torch.cat((xb, new_inds), dim=-1)  # B,T -> B,T+1
        return xb
    
bigram = BigramLM(VOCAB_SIZE)
logits, loss = bigram(xb, yb)
print(logits.shape, loss)
print(math.log(65))

new_x = torch.zeros((2, 1), dtype=torch.long)
print([decode(r) for r in bigram.generate(new_x, 10).tolist()])

# Train.
optim = AdamW(bigram.parameters(), lr=1e-3)

BATCH_SIZE = 32

for i in range(5_000):
    xb, yb = get_batch(train_data)
    _, loss = bigram(xb, yb)

    optim.zero_grad()
    loss.backward()
    optim.step()

print(loss.item())

new_x = torch.zeros((1, 1), dtype=torch.long)
print(decode(bigram.generate(new_x, 100).tolist()[0]))

torch.Size([4, 8, 65]) tensor(5.0364, grad_fn=<NllLossBackward0>)
4.174387269895637
["\nl-QYjt'CL?", '\nlcmEzyyf,O']
2.593620538711548

'-d hecuiNTha se J.eie bunthe ak.
LAt 'sca m; oivI rEG?
My ak se scaXz-Si?egrUNUCLcy m avanghtand d 


# Math behind self-attention

In [6]:
torch.manual_seed(1337)

B,T,C = 4,8,2
x = torch.randn(B,T,C)
print(x.shape)

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xbow[b,t] = torch.mean(x[b, :t+1], dim=0)

print(x[0])
print(xbow[0])

torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [7]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim=-1, keepdim=True)
print(wei)

xbow2 = wei @ x  # T,T @ B,T,C -> B, T, C
print((xbow-xbow2).abs().max())

print(xbow2[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor(3.2363e-08)
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [8]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei.masked_fill_(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x
(xbow3 - xbow).abs().max(), torch.allclose(xbow2, xbow3)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


(tensor(3.2363e-08), True)

In [9]:
torch.manual_seed(1337)

B,T,C = 4, 8, 32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

q = query(x)  # (B,T,C) -> (B,T,head_size)
k = key(x)  # (B,T,C) -> (B,T,head_size)
v = value(x)  # (B,T,C) -> (B,T,head_size)

wei = q @ k.transpose(-1, -2)  # (B,T,head_size) @ (B,head_size,T) -> (B,T,T)
print(wei.shape)

tril = torch.triu(torch.ones(T,T, dtype=torch.bool), diagonal=1)
wei.masked_fill_(tril, float("-inf"))
wei = F.softmax(wei, dim=-1)
print(wei[0])

out = wei @ v  # (B,T,T) @ (B,T,head_size) -> (B, T, head_size)
out.shape

torch.Size([4, 8, 8])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)


torch.Size([4, 8, 16])

In [10]:
a = torch.arange(15).reshape(3, 5)
print(a)

print(a.transpose(-2, -1))

T,C = a.shape
print(a.view(C,T))


tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]])
tensor([[ 0,  5, 10],
        [ 1,  6, 11],
        [ 2,  7, 12],
        [ 3,  8, 13],
        [ 4,  9, 14]])
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14]])


In [11]:
q = torch.randn(B,T,head_size)
k = torch.randn(B,T,head_size)

wei = q @ k.transpose(-2, -1)
print(wei.var())

wei *= head_size ** -0.5
print(wei.var())

tensor(16.1599)
tensor(1.0100)


# BatchNorm vs LayerNorm

In [12]:
class BatchNorm:
    def __init__(self, dim, momentum=0.1, eps=1e-5):
        self.momentum = momentum
        self.eps = eps
        self.training = True

        # Params.
        self.scale = torch.ones(dim)
        self.bias = torch.zeros(dim)

        # Buffers.
        self.running_mean = torch.zeros((1, dim))
        self.running_std = torch.ones((1, dim))

    def __call__(self, xb):
        if self.training:
            # Normalize each batch to have mean and variance for every feature.
            mean = xb.mean(0, keepdim=True)
            std = xb.std(0, keepdim=True)
            
            # Update running stats.
            with torch.no_grad():
                self.running_mean = ((1.0-self.momentum) * self.running_mean) + (self.momentum * mean)
                self.running_std = ((1.0-self.momentum) * self.running_std) + (self.momentum * std)
        else:
            mean = self.running_mean
            std = self.running_std

        normed = (xb - mean) / (std + self.eps)
        self.out = (self.scale * normed) + self.bias
        return self.out
    
    def parameters(self):
        return [self.scale, self.bias]
    
class LayerNorm:
    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.scale = torch.ones(dim)
        self.bias = torch.zeros(dim)

    def __call__(self, xb):
        # Normalize each sample individually to have mean and variance. 
        mean = xb.mean(1, keepdim=True)
        std = xb.std(1, keepdim=True)
        normed = (xb - mean) / (std + self.eps)
        self.out = (self.scale * normed) + self.bias
        return self.out
    
    def parameters(self):
        return [self.scale, self.bias]
    
x = torch.randn(32, 100)
bn = BatchNorm(100)
x_bn = bn(x)
print(x_bn[:, 0].mean(), x_bn[:, 0].std())
print(x_bn[0,:].mean(), x_bn[0,:].std())

ln = LayerNorm(100)
x_ln = ln(x)
print(x_ln[:, 0].mean(), x_ln[:, 0].std())
print(x_ln[0,:].mean(), x_ln[0,:].std())

tensor(3.7253e-08) tensor(1.0000)
tensor(0.0459) tensor(1.0197)
tensor(0.0222) tensor(0.9228)
tensor(-4.7684e-09) tensor(1.0000)


Transformer paper: https://arxiv.org/pdf/1706.03762

TODO:
* BPE tokenization
* GPT-2 repro
* encoder-decoder models (GPT is decoder only)

# Efficient Multi-Headed Attention

In [13]:
# KQV in 1 step.
torch.manual_seed(42)

B,T,C = 2, 4, 12
x = torch.randint(0, 10, (B,T,C)).float()
print(x.shape)

HEAD_SIZE = 6

qkv_linear = nn.Linear(C, HEAD_SIZE * 3, bias=False)
qkv = qkv_linear(x)
print(qkv.shape)

q, k, v = qkv.tensor_split(3, dim=-1)
print(q.shape, k.shape, v.shape)

torch.Size([2, 4, 12])
torch.Size([2, 4, 18])
torch.Size([2, 4, 6]) torch.Size([2, 4, 6]) torch.Size([2, 4, 6])


In [14]:
# Batching no concat.
B,T,C = 1, 4, 12
x = torch.randint(0, 10, (B,T,C)).float()
print("Original")
print(x.shape)
print(x)

# We want (B,T,C) -> (B,n_heads,T,head_size)
N_HEADS = 2
HEAD_SIZE = 6

x = x.view(B, T, N_HEADS, HEAD_SIZE).transpose(1, 2)
print("\nBatched")
print(x)
print(x.shape)

x = x.transpose(2, 1).contiguous().view(B,T,C)
print("\nUnbatched")
print(x)
print(x.shape)

Original
torch.Size([1, 4, 12])
tensor([[[5., 0., 8., 5., 8., 8., 6., 1., 5., 8., 1., 9.],
         [1., 1., 0., 2., 9., 1., 6., 0., 9., 6., 5., 3.],
         [0., 8., 5., 3., 4., 9., 8., 3., 4., 2., 4., 1.],
         [3., 2., 9., 8., 6., 3., 9., 4., 7., 1., 6., 4.]]])

Batched
tensor([[[[5., 0., 8., 5., 8., 8.],
          [1., 1., 0., 2., 9., 1.],
          [0., 8., 5., 3., 4., 9.],
          [3., 2., 9., 8., 6., 3.]],

         [[6., 1., 5., 8., 1., 9.],
          [6., 0., 9., 6., 5., 3.],
          [8., 3., 4., 2., 4., 1.],
          [9., 4., 7., 1., 6., 4.]]]])
torch.Size([1, 2, 4, 6])

Unbatched
tensor([[[5., 0., 8., 5., 8., 8., 6., 1., 5., 8., 1., 9.],
         [1., 1., 0., 2., 9., 1., 6., 0., 9., 6., 5., 3.],
         [0., 8., 5., 3., 4., 9., 8., 3., 4., 2., 4., 1.],
         [3., 2., 9., 8., 6., 3., 9., 4., 7., 1., 6., 4.]]])
torch.Size([1, 4, 12])


In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, n_heads):
        super().__init__()
        self.n_heads = n_heads
        assert embed_dim % n_heads == 0, f"{embed_dim=} must be a multiple of {n_heads=}"
        self.head_size = embed_dim // n_heads
        self.qkv_linear = nn.Linear(self.head_size, self.head_size*3, bias=False)
        self.register_buffer(
            "tril", torch.triu(torch.ones(BLOCK_SIZE, BLOCK_SIZE, dtype=torch.bool), diagonal=1)
        )

    def forward(self, x):
        B,T,C = x.shape
        # Batch heads: (B,T,C) -> (B,n_heads,T,head_size).
        x = x.view(B, T, self.n_heads, self.head_size).transpose(1, 2)

        qkv = self.qkv_linear(x)
        # Split QKV: (B,nh,T,hs*3) -> 3*(B,nh,T,hs).
        q, k, v = qkv.tensor_split(3, dim=-1)

        wei = q @ k.transpose(-1, -2)
        wei.masked_fill_(self.tril[:T,:T], float("-inf"))
        wei = F.softmax(wei / (self.head_size**0.5), dim=-1)

        out = wei @ v
        # Unbatch heads: (B,nh,T,hs) -> (B,T,C).
        out = out.transpose(1,2).contiguous().view(B,T,C)
        return out
    
B,T,C = 2, 4, 24
x = torch.randn(B,T,C).float()
print(x.shape)

mha = MultiHeadAttention(embed_dim=C, n_heads=8)
print(mha(x).shape)

torch.Size([2, 4, 24])
torch.Size([2, 4, 24])
